In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("shusrith/machine-trainslation")

# print("Path to dataset files:", path)

In [2]:
# !ls $path/EnglishOrSpanish

In [3]:
import pandas as pd
from sklearn.utils import shuffle
df = pd.read_csv(f"EnglishOrSpanish/output_joint1.csv")
df = shuffle(df)
df

,English,Spanish
58526,"[2, 1185, 300, 1609, 391, 300, 2101, 224, 3, 0...","[2, 6133, 321, 1622, 1086, 321, 2113, 224, 3, ..."
61040,"[2, 449, 304, 3230, 362, 448, 1105, 224, 3, 0,...","[2, 393, 4462, 294, 352, 1121, 224, 3, 0, 0, 0..."
55871,"[2, 426, 1664, 362, 1903, 334, 404, 224, 3, 0,...","[2, 444, 1834, 294, 1791, 318, 589, 224, 3, 0,..."
103847,"[2, 568, 449, 989, 904, 370, 431, 498, 328, 22...","[2, 664, 1875, 1702, 265, 11069, 302, 1407, 60..."
72195,"[2, 300, 1066, 5955, 328, 272, 402, 804, 224, ...","[2, 331, 1233, 328, 4065, 1207, 644, 224, 3, 0..."
...,...,...
80241,"[2, 442, 996, 272, 300, 1827, 722, 2735, 224, ...","[2, 463, 620, 322, 2735, 355, 1827, 224, 3, 0,..."
9001,"[2, 270, 498, 370, 1066, 224, 3, 0, 0, 0, 0, 0...","[2, 328, 731, 3318, 1233, 224, 3, 0, 0, 0, 0, ..."
62406,"[2, 270, 1167, 265, 6204, 976, 300, 1366, 224,...","[2, 4329, 410, 8242, 4833, 294, 321, 1429, 224..."
54914,"[2, 538, 826, 465, 473, 611, 361, 224, 3, 0, 0...","[2, 11376, 302, 6470, 224, 3, 0, 0, 0, 0, 0, 0..."


In [4]:
import ast

df["English"] = df["English"].apply(ast.literal_eval)
df["Spanish"] = df["Spanish"].apply(ast.literal_eval)
df

,English,Spanish
58526,"[2, 1185, 300, 1609, 391, 300, 2101, 224, 3, 0...","[2, 6133, 321, 1622, 1086, 321, 2113, 224, 3, ..."
61040,"[2, 449, 304, 3230, 362, 448, 1105, 224, 3, 0,...","[2, 393, 4462, 294, 352, 1121, 224, 3, 0, 0, 0..."
55871,"[2, 426, 1664, 362, 1903, 334, 404, 224, 3, 0,...","[2, 444, 1834, 294, 1791, 318, 589, 224, 3, 0,..."
103847,"[2, 568, 449, 989, 904, 370, 431, 498, 328, 22...","[2, 664, 1875, 1702, 265, 11069, 302, 1407, 60..."
72195,"[2, 300, 1066, 5955, 328, 272, 402, 804, 224, ...","[2, 331, 1233, 328, 4065, 1207, 644, 224, 3, 0..."
...,...,...
80241,"[2, 442, 996, 272, 300, 1827, 722, 2735, 224, ...","[2, 463, 620, 322, 2735, 355, 1827, 224, 3, 0,..."
9001,"[2, 270, 498, 370, 1066, 224, 3, 0, 0, 0, 0, 0...","[2, 328, 731, 3318, 1233, 224, 3, 0, 0, 0, 0, ..."
62406,"[2, 270, 1167, 265, 6204, 976, 300, 1366, 224,...","[2, 4329, 410, 8242, 4833, 294, 321, 1429, 224..."
54914,"[2, 538, 826, 465, 473, 611, 361, 224, 3, 0, 0...","[2, 11376, 302, 6470, 224, 3, 0, 0, 0, 0, 0, 0..."


In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset

train = TensorDataset(
    torch.tensor(df["English"][:100000].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][:100000].tolist(), dtype=torch.long),
)
test = TensorDataset(
    torch.tensor(df["English"][100000:].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][100000:].tolist(), dtype=torch.long),
)
train_loader = DataLoader(train, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test, batch_size=256, shuffle=False, drop_last=True)

In [6]:
import json
with open(f"EnglishOrSpanish/vocab.json", "r") as f:
    vocab = json.load(f)

In [7]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=2, dropout=0.3):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    nn.init.zeros_(param)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(embedded)
        return output, hidden, cell

In [8]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=2, dropout=0.3):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    nn.init.zeros_(param)

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(1)
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        output = self.fc(output).squeeze(1)
        return output, hidden, cell

In [9]:
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, teacher_forcing_ratio):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.vocab_size = self.decoder.vocab_size
        self.teacher_forcing_ratio = teacher_forcing_ratio

    def forward(self, source, target):
        _, hidden, cell = self.encoder(source)
        x = target[:, 0]
        outputs = torch.zeros(target.shape[0], target.shape[1], self.vocab_size).to(self.device)
        for t in range(1, target.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell)
            outputs[:, t] = output
            teacher_force = random.random() < self.teacher_forcing_ratio
            top1 = output.argmax(1)
            x = target[:, t] if teacher_force else top1
    
        return outputs
    
    def predict(self, source):
        _, hidden, cell = self.encoder(source)
        x = source[:, 0]
        outputs = torch.zeros(source.shape[0], source.shape[1], self.vocab_size).to(self.device)
        for t in range(1, source.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell)
            outputs[:, t] = output
            top1 = output.argmax(1)
            x = top1
    
        return outputs

In [10]:
def train(
        model,
        train_loader,
        test_loader,
        optimizer,
        criterion,
        device,
        scheduler,
        num_epochs,
    ):
        vocab_size = model.decoder.vocab_size
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            model.train()
            epoch_loss = 0
            progress_bar = tqdm(train_loader, desc="Training", leave=False)

            for src, trg in progress_bar:
                src, trg = src.to(device), trg.to(device)

                batch_size, trg_len = trg.shape

                optimizer.zero_grad()

                outputs = model(src, trg)
                loss = criterion(
                    outputs[:, 1:].reshape(-1, vocab_size), trg[:, 1:].reshape(-1)
                )
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()

                progress_bar.set_postfix(
                    loss=f"{loss.item():.4f}"
                )

            train_loss = epoch_loss / len(train_loader)

            # ---------------- VALIDATION ---------------- #
            model.eval()
            val_epoch_loss = 0
            val_epoch_acc = 0
            progress_bar = tqdm(test_loader, desc="Validating", leave=False)

            with torch.no_grad():
                for src, trg in progress_bar:
                    src, trg = src.to(device), trg.to(device)
                    outputs = model(src, trg)
                    loss = criterion(
                        outputs[:, 1:].reshape(-1, vocab_size),
                        trg[:, 1:].reshape(-1),
                    )
                    val_epoch_loss += loss.item()

                    progress_bar.set_postfix(
                        loss=f"{loss.item():.4f}"
                    )

            val_loss = val_epoch_loss / len(test_loader)
            print(
                f"Train loss : {train_loss}, Val_loss : {val_loss}"
            )
            scheduler.step(val_loss)
        return train_loss, val_loss

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)

embedding_dim = 384
hidden_size = 64
encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, device, 0.5).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(encoder.parameters(), lr=0.05, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

train(seq2seq, train_loader, test_loader, optimizer, criterion, device, scheduler, 30)


Epoch 1/30


Train loss : 10.227347498673659, Val_loss : 10.196668515163186

Epoch 2/30


KeyboardInterrupt: 

: 

In [ ]:
torch.save(seq2seq.state_dict(), "seq2seq-noatten-bidir.pth")